# 🔧 Development Workflow: KubeSentiment Development & Testing

This notebook explores the development workflow for KubeSentiment, including testing strategies, code quality, CI/CD integration, and best practices for MLOps development.

## 🎯 Learning Objectives

By the end of this notebook, you will:
1. Understand the testing pyramid and test coverage
2. Learn about code quality tools and automation
3. Explore CI/CD pipeline integration
4. Understand development best practices
5. Learn debugging and troubleshooting techniques
6. Master the development workflow from idea to production

## 📦 Setup and Dependencies

First, let's install the required dependencies and set up our environment.

In [ ]:
# Install required packages for this notebook
# Note: This cell might take a few minutes to run
!pip install -r ../requirements.txt

### ✅ Version Check
Let's check the versions of the installed libraries to ensure our environment is reproducible.

In [ ]:
# List installed packages to ensure reproducibility
!pip list

## 🧪 Testing Pyramid & Strategy

### The Testing Pyramid

```
     /\
    /  \
   /Unit\
  /______\
 /Integration\
/____________\
     E2E
```

### KubeSentiment Testing Strategy

- **Unit Tests**: Test individual functions and classes
- **Integration Tests**: Test API endpoints and service interactions
- **End-to-End Tests**: Test complete user workflows
- **Performance Tests**: Load testing and benchmarking
- **Security Tests**: Vulnerability scanning and validation

In [ ]:
# Setup and imports
import os
import sys
import subprocess
import pytest
import coverage
import json
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('default')
sns.set_palette("husl")

# Define paths
PROJECT_ROOT = Path("../../")
APP_DIR = PROJECT_ROOT / "app"
TESTS_DIR = PROJECT_ROOT / "tests"

print("✅ Libraries imported successfully!")
print(f"📁 Project root: {PROJECT_ROOT.absolute()}")
print(f"🧪 Tests directory: {TESTS_DIR.absolute()}")

## 📊 Test Coverage Analysis

Let's analyze the current test coverage and identify areas for improvement.

In [ ]:
# Run tests and analyze coverage
def run_test_suite():
    """Run the complete test suite and collect results."""
    
    print("🧪 Running Test Suite...")
    print("=" * 50)
    
    # Change to project root
    os.chdir(PROJECT_ROOT)
    
    try:
        # Run pytest with coverage
        result = subprocess.run([
            sys.executable, '-m', 'pytest', 
            'tests/', 
            '-v', 
            '--tb=short',
            '--cov=app',
            '--cov-report=json',
            '--cov-report=term-missing'
        ], capture_output=True, text=True, timeout=300)
        
        print("📊 Test Results:")
        print(result.stdout)
        
        if result.stderr:
            print("⚠️ Errors/Warnings:")
            print(result.stderr)
        
        # Check if coverage report was generated
        coverage_file = Path("coverage.json")
        if coverage_file.exists():
            with open(coverage_file, 'r') as f:
                coverage_data = json.load(f)
            
            # Extract summary
            totals = coverage_data.get('totals', {})
            covered_lines = totals.get('covered_lines', 0)
            num_statements = totals.get('num_statements', 1)
            coverage_percent = totals.get('percent_covered', 0)
            
            return {
                "success": result.returncode == 0,
                "return_code": result.returncode,
                "stdout": result.stdout,
                "stderr": result.stderr,
                "coverage": {
                    "covered_lines": covered_lines,
                    "total_lines": num_statements,
                    "percentage": coverage_percent
                },
                "coverage_data": coverage_data
            }
        else:
            return {
                "success": False,
                "error": "Coverage report not generated",
                "stdout": result.stdout,
                "stderr": result.stderr
            }
            
    except subprocess.TimeoutExpired:
        return {"success": False, "error": "Test execution timed out"}
    except Exception as e:
        return {"success": False, "error": str(e)}

# Run tests if in the correct environment
if PROJECT_ROOT.exists() and TESTS_DIR.exists():
    test_results = run_test_suite()
    
    if test_results["success"]:
        print("✅ Test suite completed successfully!")
        
        if "coverage" in test_results:
            cov = test_results["coverage"]
            print(f"📊 Code Coverage: {cov['percentage']:.1f}%")
            print(f"   📝 Covered Lines: {cov['covered_lines']}")
            print(f"   📏 Total Lines: {cov['total_lines']}")
    else:
        print("❌ Test suite failed")
        print(f"🔍 Error: {test_results.get('error', 'Unknown error')}")
        
        # Show some output for debugging
        stdout = test_results.get('stdout', '')
        if stdout:
            print("\n📄 Test Output (first 500 chars):")
            print(stdout[:500] + "..." if len(stdout) > 500 else stdout)

else:
    print("⏭️ Skipping test execution - not in project environment")
    print("💡 To run tests, execute this notebook from the project root directory")
    
    # Create sample test results for demonstration
    test_results = {
        "success": True,
        "coverage": {
            "covered_lines": 850,
            "total_lines": 1200,
            "percentage": 70.8
        }
    }

## 📈 Code Quality Analysis

Let's analyze code quality using various linting and formatting tools.

In [ ]:
# Code quality analysis
def analyze_code_quality():
    """Run code quality analysis tools."""
    
    quality_results = {}
    
    # Change to project root
    os.chdir(PROJECT_ROOT)
    
    # List of tools to check
    tools = [
        {
            "name": "black",
            "command": ["black", "--check", "--diff", "app/", "tests/"],
            "description": "Code formatting check"
        },
        {
            "name": "isort",
            "command": ["isort", "--check-only", "--diff", "app/", "tests/"],
            "description": "Import sorting check"
        },
        {
            "name": "ruff",
            "command": ["ruff", "check", "app/", "tests/"],
            "description": "Fast Python linter"
        },
        {
            "name": "mypy",
            "command": ["mypy", "app/", "--ignore-missing-imports"],
            "description": "Static type checking"
        },
        {
            "name": "flake8",
            "command": ["flake8", "app/", "tests/", "--max-line-length=88", "--extend-ignore=E203,W503"],
            "description": "Style guide enforcement"
        }
    ]
    
    for tool in tools:
        print(f"🔍 Running {tool['name']} - {tool['description']}")
        
        try:
            result = subprocess.run(
                tool["command"],
                capture_output=True,
                text=True,
                timeout=60
            )
            
            quality_results[tool["name"]] = {
                "success": result.returncode == 0,
                "return_code": result.returncode,
                "stdout": result.stdout,
                "stderr": result.stderr,
                "description": tool["description"]
            }
            
            if result.returncode == 0:
                print(f"   ✅ PASSED")
            else:
                print(f"   ❌ FAILED (exit code: {result.returncode})")
                
                # Show first few lines of output
                output = result.stdout + result.stderr
                if output:
                    lines = output.split('\n')[:5]
                    for line in lines:
                        if line.strip():
                            print(f"      {line}")
                    
        except FileNotFoundError:
            print(f"   ⚠️ {tool['name']} not installed")
            quality_results[tool["name"]] = {
                "success": False,
                "error": "Tool not installed",
                "description": tool["description"]
            }
        except subprocess.TimeoutExpired:
            print(f"   ⏱️ {tool['name']} timed out")
            quality_results[tool["name"]] = {
                "success": False,
                "error": "Timeout",
                "description": tool["description"]
            }
        except Exception as e:
            print(f"   💥 {tool['name']} error: {e}")
            quality_results[tool["name"]] = {
                "success": False,
                "error": str(e),
                "description": tool["description"]
            }
        
        print()
    
    return quality_results

# Run code quality analysis
print("🔧 Code Quality Analysis:")
print("=" * 50)

quality_results = analyze_code_quality()

# Summarize results
total_tools = len(quality_results)
passed_tools = sum(1 for r in quality_results.values() if r.get("success", False))

print(f"📊 Code Quality Summary: {passed_tools}/{total_tools} tools passed")

# Visualize results
fig, ax = plt.subplots(figsize=(10, 6))

tools = list(quality_results.keys())
passed = [1 if r.get("success", False) else 0 for r in quality_results.values()]
failed = [1 if not r.get("success", False) else 0 for r in quality_results.values()]

bar1 = ax.bar(tools, passed, label='Passed', color='green', alpha=0.7)
bar2 = ax.bar(tools, failed, bottom=passed, label='Failed', color='red', alpha=0.7)

ax.set_ylabel('Status')
ax.set_title('Code Quality Tool Results')
ax.set_xticklabels(tools, rotation=45, ha='right')
ax.legend()
ax.grid(True, alpha=0.3)

# Add value labels
for bar, status in zip(bar1, passed):
    if status > 0:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height/2, '✓', 
                ha='center', va='center', color='white', fontweight='bold')

plt.tight_layout()
plt.show()

# Show detailed results for failed tools
failed_tools = [name for name, result in quality_results.items() if not result.get("success", False)]
if failed_tools:
    print("\n❌ Failed Tools Details:")
    for tool_name in failed_tools:
        result = quality_results[tool_name]
        print(f"\n🔧 {tool_name} ({result.get('description', 'Unknown')}):")
        if 'error' in result:
            print(f"   Error: {result['error']}")
        else:
            # Show first few lines of output
            output = result.get('stdout', '') + result.get('stderr', '')
            lines = output.split('\n')[:10]
            for line in lines:
                if line.strip():
                    print(f"   {line}")
else:
    print("\n✅ All code quality checks passed!")

## 🔄 CI/CD Pipeline Analysis

Let's examine the CI/CD pipeline configuration and understand the automated testing workflow.

In [ ]:
# CI/CD pipeline analysis
def analyze_ci_cd_pipeline():
    """Analyze the CI/CD pipeline configuration."""
    
    pipeline_info = {}
    
    # Analyze GitHub Actions workflow
    workflow_file = PROJECT_ROOT / ".github" / "workflows" / "ci.yml"
    
    if workflow_file.exists():
        try:
            import yaml
            with open(workflow_file, 'r') as f:
                workflow = yaml.safe_load(f)
            
            pipeline_info['github_actions'] = {
                "name": workflow.get('name', 'Unknown'),
                "triggers": workflow.get('on', {}),
                "jobs": list(workflow.get('jobs', {}).keys()),
                "python_version": workflow.get('env', {}).get('PYTHON_VERSION', 'Unknown'),
                "registry": workflow.get('env', {}).get('REGISTRY', 'Unknown')
            }
            
            # Analyze jobs
            jobs_info = []
            for job_name, job_config in workflow.get('jobs', {}).items():
                steps = job_config.get('steps', [])
                jobs_info.append({
                    "name": job_name,
                    "runs_on": job_config.get('runs-on', 'Unknown'),
                    "steps_count": len(steps),
                    "key_steps": [step.get('name', 'Unknown') for step in steps[:5]]
                })
            
            pipeline_info['jobs_detail'] = jobs_info
            
        except ImportError:
            pipeline_info['github_actions'] = {"error": "PyYAML not available"}
        except Exception as e:
            pipeline_info['github_actions'] = {"error": str(e)}
    else:
        pipeline_info['github_actions'] = {"error": "Workflow file not found"}
    
    # Check for other CI/CD files
    other_files = [
        PROJECT_ROOT / ".travis.yml",
        PROJECT_ROOT / "Jenkinsfile",
        PROJECT_ROOT / ".gitlab-ci.yml",
        PROJECT_ROOT / "Makefile"
    ]
    
    pipeline_info['other_ci_files'] = []
    for file_path in other_files:
        if file_path.exists():
            pipeline_info['other_ci_files'].append({
                "file": file_path.name,
                "exists": True,
                "size": file_path.stat().st_size
            })
    
    # Check for Docker-related files
    docker_files = [
        PROJECT_ROOT / "Dockerfile",
        PROJECT_ROOT / "docker-compose.yml",
        PROJECT_ROOT / ".dockerignore"
    ]
    
    pipeline_info['docker_files'] = []
    for file_path in docker_files:
        if file_path.exists():
            pipeline_info['docker_files'].append({
                "file": file_path.name,
                "exists": True,
                "size": file_path.stat().st_size
            })
    
    return pipeline_info

# Analyze CI/CD pipeline
print("🔄 CI/CD Pipeline Analysis:")
print("=" * 50)

pipeline_info = analyze_ci_cd_pipeline()

# Display GitHub Actions info
if 'github_actions' in pipeline_info:
    ga = pipeline_info['github_actions']
    if 'error' not in ga:
        print("🤖 GitHub Actions Workflow:")
        print(f"   📝 Name: {ga['name']}")
        print(f"   🐍 Python Version: {ga['python_version']}")
        print(f"   📦 Container Registry: {ga['registry']}")
        print(f"   🔄 Jobs: {', '.join(ga['jobs'])}")
        
        # Triggers
        triggers = ga['triggers']
        print(f"   🎯 Triggers:")
        if 'push' in triggers:
            push = triggers['push']
            if 'branches' in push:
                print(f"      Push to branches: {', '.join(push['branches'])}")
        if 'pull_request' in triggers:
            pr = triggers['pull_request']
            if 'branches' in pr:
                print(f"      PR to branches: {', '.join(pr['branches'])}")
        
        # Job details
        if 'jobs_detail' in pipeline_info:
            print(f"\n📋 Job Details:")
            for job in pipeline_info['jobs_detail']:
                print(f"   🔧 {job['name']} ({job['runs_on']}):")
                print(f"      Steps: {job['steps_count']}")
                print(f"      Key steps: {', '.join(job['key_steps'][:3])}")
    else:
        print(f"❌ GitHub Actions error: {ga['error']}")

# Display other CI/CD files
other_files = pipeline_info.get('other_ci_files', [])
if other_files:
    print(f"\n📁 Other CI/CD Files:")
    for file_info in other_files:
        print(f"   📄 {file_info['file']} ({file_info['size']} bytes)")

# Display Docker files
docker_files = pipeline_info.get('docker_files', [])
if docker_files:
    print(f"\n🐳 Docker Files:")
    for file_info in docker_files:
        print(f"   📦 {file_info['file']} ({file_info['size']} bytes)")

# Create pipeline visualization
if 'jobs_detail' in pipeline_info:
    fig, ax = plt.subplots(figsize=(10, 6))
    
    jobs = pipeline_info['jobs_detail']
    job_names = [job['name'] for job in jobs]
    step_counts = [job['steps_count'] for job in jobs]
    
    bars = ax.bar(job_names, step_counts, color='skyblue', alpha=0.7)
    ax.set_ylabel('Number of Steps')
    ax.set_title('CI/CD Pipeline Complexity')
    ax.grid(True, alpha=0.3)
    
    # Add value labels
    for bar, count in zip(bars, step_counts):
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height + 0.1, str(count), 
                ha='center', va='bottom')
    
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

print("\n✅ CI/CD Analysis Complete!")
print("\n💡 Key Insights:")
print("   • Automated testing on multiple branches")
print("   • Multi-stage pipeline: test → build → deploy")
print("   • Security scanning integrated")
print("   • Multi-environment deployment support")

## 🐛 Debugging and Troubleshooting

Let's explore common debugging techniques and troubleshooting workflows.

In [ ]:
# Debugging and troubleshooting utilities
def analyze_common_issues():
    """Analyze common development and deployment issues."""
    
    issues_analysis = {
        "import_errors": {
            "symptoms": ["ModuleNotFoundError", "ImportError", "No module named"],
            "causes": ["Missing dependencies", "Virtual environment not activated", "PYTHONPATH issues"],
            "solutions": [
                "pip install -r requirements.txt",
                "source venv/bin/activate (Linux/Mac)",
                "venv\\Scripts\\activate (Windows)",
                "Check PYTHONPATH environment variable"
            ]
        },
        "model_loading_failures": {
            "symptoms": ["ModelNotLoadedError", "Failed to load model", "HuggingFace error"],
            "causes": ["Network connectivity", "Invalid model name", "Cache directory issues"],
            "solutions": [
                "Check internet connection",
                "Verify model name in settings",
                "Clear model cache: rm -rf ~/.cache/huggingface",
                "Set HF_HUB_OFFLINE=1 for offline mode"
            ]
        },
        "api_connection_errors": {
            "symptoms": ["Connection refused", "Timeout", "502 Bad Gateway"],
            "causes": ["Service not running", "Wrong port", "Network issues"],
            "solutions": [
                "Check if service is running: docker ps",
                "Verify port configuration",
                "Check service logs: docker logs <container>",
                "Test connectivity: curl http://localhost:8000/health"
            ]
        },
        "performance_issues": {
            "symptoms": ["Slow responses", "High latency", "Memory errors"],
            "causes": ["Resource constraints", "Inefficient code", "Cache misses"],
            "solutions": [
                "Monitor resource usage",
                "Profile code performance",
                "Optimize cache settings",
                "Consider model optimization (ONNX)"
            ]
        },
        "test_failures": {
            "symptoms": ["Test failures", "Assertion errors", "Import errors in tests"],
            "causes": ["Code changes", "Environment differences", "Mock setup issues"],
            "solutions": [
                "Run tests locally first",
                "Check test dependencies",
                "Update test mocks and fixtures",
                "Review recent code changes"
            ]
        }
    }
    
    return issues_analysis

# Create troubleshooting guide
def create_troubleshooting_workflow():
    """Create a systematic troubleshooting workflow."""
    
    workflow = {
        "step_1_reproduce": {
            "title": "Reproduce the Issue",
            "actions": [
                "Gather error messages and logs",
                "Note exact steps to reproduce",
                "Record environment details (OS, Python version, etc.)",
                "Try to isolate the problem"
            ]
        },
        "step_2_check_basics": {
            "title": "Check Basic Functionality",
            "actions": [
                "Verify service is running",
                "Check network connectivity",
                "Validate configuration",
                "Test with minimal example"
            ]
        },
        "step_3_analyze_logs": {
            "title": "Analyze Logs and Metrics",
            "actions": [
                "Check application logs",
                "Review monitoring dashboards",
                "Look for error patterns",
                "Compare with working baseline"
            ]
        },
        "step_4_isolate_cause": {
            "title": "Isolate the Root Cause",
            "actions": [
                "Use binary search to isolate changes",
                "Test individual components",
                "Check recent deployments/changes",
                "Review configuration differences"
            ]
        },
        "step_5_implement_fix": {
            "title": "Implement and Test Fix",
            "actions": [
                "Develop minimal fix",
                "Test fix thoroughly",
                "Verify no regressions",
                "Document the solution"
            ]
        },
        "step_6_prevent_future": {
            "title": "Prevent Future Issues",
            "actions": [
                "Add monitoring/alerts",
                "Improve error handling",
                "Update documentation",
                "Add regression tests"
            ]
        }
    }
    
    return workflow

# Display troubleshooting information
print("🐛 Debugging and Troubleshooting Guide:")
print("=" * 60)

# Show common issues
issues = analyze_common_issues()
print("🔧 Common Issues and Solutions:")
print("\n" + "=" * 50)

for issue_name, issue_info in issues.items():
    print(f"\n🚨 {issue_name.replace('_', ' ').title()}:")
    print(f"   📝 Symptoms: {', '.join(issue_info['symptoms'])}")
    print(f"   🔍 Causes: {', '.join(issue_info['causes'])}")
    print(f"   ✅ Solutions:")
    for solution in issue_info['solutions']:
        print(f"      • {solution}")

# Show troubleshooting workflow
workflow = create_troubleshooting_workflow()
print(f"\n\n🔄 Systematic Troubleshooting Workflow:")
print("=" * 50)

for step_key, step_info in workflow.items():
    step_num = step_key.split('_')[1]
    print(f"\n{step_num}. {step_info['title']}:")
    for action in step_info['actions']:
        print(f"   • {action}")

# Create debugging tools
def debug_api_endpoint(url: str, timeout: int = 10):
    """Debug API endpoint connectivity."""
    
    import requests
    
    debug_info = {
        "url": url,
        "timestamp": pd.Timestamp.now(),
        "checks": {}
    }
    
    # Basic connectivity check
    try:
        response = requests.get(url, timeout=timeout)
        debug_info["checks"]["connectivity"] = {
            "status": "success",
            "status_code": response.status_code,
            "response_time_ms": response.elapsed.total_seconds() * 1000
        }
    except requests.exceptions.ConnectionError:
        debug_info["checks"]["connectivity"] = {
            "status": "failed",
            "error": "Connection refused - service may not be running"
        }
    except requests.exceptions.Timeout:
        debug_info["checks"]["connectivity"] = {
            "status": "failed",
            "error": "Timeout - service may be overloaded"
        }
    except Exception as e:
        debug_info["checks"]["connectivity"] = {
            "status": "failed",
            "error": str(e)
        }
    
    return debug_info

# Test debugging tool
print(f"\n🔍 Testing API Debugging Tool:")
debug_result = debug_api_endpoint("http://localhost:8000/health")

print(f"🌐 URL: {debug_result['url']}")
print(f"🕒 Timestamp: {debug_result['timestamp']}")

for check_name, check_result in debug_result['checks'].items():
    status = check_result['status']
    emoji = "✅" if status == "success" else "❌"
    print(f"{emoji} {check_name.title()}: {status.upper()}")
    
    if status == "success":
        if "status_code" in check_result:
            print(f"   Status Code: {check_result['status_code']}")
        if "response_time_ms" in check_result:
            print(f"   Response Time: {check_result['response_time_ms']:.1f}ms")
    else:
        print(f"   Error: {check_result['error']}")

print("\n💡 Pro Tip: Use this systematic approach to debug issues efficiently!")

## 🧪 Automated Testing

We can integrate automated tests directly into our notebooks using `pytest`.

In [ ]:
# Create a simple test file
test_code = """
import os

def test_project_structure():
    # Test that the project structure is as expected
    assert os.path.isdir('app'), \"'app' directory should exist\"
    assert os.path.isdir('tests'), \"'tests' directory should exist\"

def test_config_files():
    # Test that the main configuration files exist
    assert os.path.isfile('config/config.yaml'), \"'config.yaml' should exist\"
"""
with open("test_workflow.py", "w") as f:
    f.write(test_code)

# Run pytest
!pytest test_workflow.py -v

## 🚀 Development Workflow Best Practices

Let's explore the complete development workflow from idea to production.

In [ ]:
# Development workflow visualization
def create_development_workflow():
    """Create a comprehensive development workflow guide."""
    
    workflow = {
        "planning": {
            "title": "Planning & Design",
            "duration": "1-2 days",
            "activities": [
                "Define requirements and success criteria",
                "Design API endpoints and data models",
                "Plan testing strategy and coverage goals",
                "Create implementation plan and milestones"
            ],
            "deliverables": [
                "Technical specification document",
                "API design document",
                "Test plan",
                "Project timeline"
            ]
        },
        "development": {
            "title": "Local Development",
            "duration": "3-7 days",
            "activities": [
                "Set up local development environment",
                "Implement core functionality",
                "Write comprehensive unit tests",
                "Implement logging and error handling",
                "Create integration tests",
                "Performance testing and optimization"
            ],
            "tools": [
                "VS Code / PyCharm",
                "Docker for local services",
                "pytest for testing",
                "black/isort for code formatting",
                "pre-commit hooks"
            ]
        },
        "code_quality": {
            "title": "Code Quality & Review",
            "duration": "1-2 days",
            "activities": [
                "Run automated code quality checks",
                "Achieve target test coverage (>80%)",
                "Security scanning and vulnerability assessment",
                "Code review and feedback incorporation",
                "Documentation updates"
            ],
            "gates": [
                "All tests passing",
                "Code quality checks passed",
                "Security scan clean",
                "Documentation updated"
            ]
        },
        "ci_cd": {
            "title": "CI/CD Pipeline",
            "duration": "1-4 hours",
            "activities": [
                "Automated testing on multiple environments",
                "Code quality validation",
                "Security scanning",
                "Container building and deployment",
                "Integration testing"
            ],
            "benefits": [
                "Consistent deployment process",
                "Automated quality gates",
                "Fast feedback on issues",
                "Reliable release process"
            ]
        },
        "deployment": {
            "title": "Production Deployment",
            "duration": "2-4 hours",
            "activities": [
                "Deploy to staging environment",
                "Integration and end-to-end testing",
                "Performance validation",
                "Security assessment",
                "Production deployment with rollback plan",
                "Post-deployment monitoring"
            ],
            "environments": [
                "Development (dev)",
                "Staging (staging)",
                "Production (prod)"
            ]
        },
        "monitoring": {
            "title": "Monitoring & Maintenance",
            "duration": "Ongoing",
            "activities": [
                "Monitor system health and performance",
                "Track business metrics and KPIs",
                "Respond to alerts and incidents",
                "Regular security updates",
                "Performance optimization",
                "User feedback collection"
            ],
            "tools": [
                "Prometheus + Grafana",
                "ELK stack for logging",
                "Alertmanager for notifications",
                "Custom dashboards"
            ]
        }
    }
    
    return workflow

# Display development workflow
print("🚀 Development Workflow Best Practices:")
print("=" * 60)

workflow = create_development_workflow()

for phase_key, phase_info in workflow.items():
    print(f"\n📋 {phase_info['title']} ({phase_info['duration']})")
    print("-" * 50)
    
    if 'activities' in phase_info:
        print("Activities:")
        for activity in phase_info['activities']:
            print(f"  • {activity}")
    
    if 'deliverables' in phase_info:
        print("\nDeliverables:")
        for deliverable in phase_info['deliverables']:
            print(f"  • {deliverable}")
    
    if 'tools' in phase_info:
        print("\nTools:")
        for tool in phase_info['tools']:
            print(f"  • {tool}")
    
    if 'gates' in phase_info:
        print("\nQuality Gates:")
        for gate in phase_info['gates']:
            print(f"  • {gate}")
    
    if 'benefits' in phase_info:
        print("\nBenefits:")
        for benefit in phase_info['benefits']:
            print(f"  • {benefit}")
    
    if 'environments' in phase_info:
        print("\nEnvironments:")
        for env in phase_info['environments']:
            print(f"  • {env}")

# Create workflow timeline visualization
fig, ax = plt.subplots(figsize=(12, 8))

# Workflow phases and estimated durations (in days)
phases = list(workflow.keys())
durations = [2, 5, 1.5, 0.25, 0.5, 30]  # Rough estimates in days
phase_names = [workflow[phase]['title'] for phase in phases]

# Create timeline
colors = ['lightblue', 'lightgreen', 'orange', 'red', 'purple', 'gray']
bars = ax.barh(phases, durations, color=colors, alpha=0.7)

ax.set_xlabel('Duration (days)')
ax.set_title('Development Workflow Timeline')
ax.set_yticks(range(len(phases)))
ax.set_yticklabels(phase_names)
ax.grid(True, alpha=0.3)

# Add duration labels
for bar, duration in zip(bars, durations):
    width = bar.get_width()
    ax.text(width + 0.1, bar.get_y() + bar.get_height()/2, 
            f'{duration}d', ha='left', va='center')

# Add cumulative timeline
cumulative = 0
for i, duration in enumerate(durations):
    cumulative += duration
    ax.axvline(cumulative, color='black', linestyle='--', alpha=0.3)
    if i < len(durations) - 1:
        ax.text(cumulative + 0.1, len(durations)-1, f'~{cumulative:.1f}d', 
                ha='left', va='top', fontsize=8)

plt.tight_layout()
plt.show()

# Quality metrics summary
quality_metrics = {
    "Code Coverage": "85%+ (target 90%+)",
    "Cyclomatic Complexity": "< 10 per function",
    "Technical Debt": "< 5% of codebase",
    "Test Execution Time": "< 5 minutes",
    "Security Vulnerabilities": "0 critical/high",
    "Performance Regression": "< 10% degradation",
    "Mean Time To Recovery": "< 1 hour",
    "Change Failure Rate": "< 15%"
}

print("\n📊 Quality Metrics Targets:")
print("=" * 40)
for metric, target in quality_metrics.items():
    print(f"{metric:<25}: {target}")

print("\n🎯 Development Workflow Complete!")
print("\n💡 Key Takeaways:")
print("   • Follow systematic development process")
print("   • Maintain high code quality standards")
print("   • Implement comprehensive testing")
print("   • Use CI/CD for automated quality gates")
print("   • Monitor and iterate on performance")
print("   • Document everything thoroughly")
print("\n🚀 Ready to deploy to production? Check the next notebook!")